---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [14]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    with open("City data/university_towns.txt", 'r') as file:
        data = []
        for line in file:
            data.append(line[:-1])  # remove new line "\n"
            
    state_town = []
    for i in range(len(data)):
        if '[edit]' in data[i]:
            state = data[i].split('[edit]')[0]
        elif '(' in data[i]:
            town = data[i].split(' (')[0]
            state_town.append([state, town])
        else:
            town = data[i]
            state_town.append([state, town])
            
    university_towns = pd.DataFrame(state_town, columns = ['State', 'Region Name'])
    return university_towns

In [15]:
get_list_of_university_towns()

,State,Region Name
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [45]:
#pd.read_excel("City data/gdplev.xls", skiprows = 7, usecols = [0,1,2,4,5,6]).rename(
#    columns = {'Unnamed: 0':'Year', 'Unnamed: 1':'Annual GDP', "Unnamed: 2":'Annual GDP based on 2009', 
#               'Unnamed: 4':'Year & Q', 'Unnamed: 5':'Quarter GDP', 'Unnamed: 6':'Quarter GDP based on 2009'}).head()

In [6]:
def get_gdp():
    gdp = pd.read_excel("City data/gdplev.xls", skiprows = 7, usecols = [4,5,6]).rename(
        columns = {'Unnamed: 4':'Q', 
                   'Unnamed: 5':'Quarterly GDP(bn)', 
                   'Unnamed: 6':'GDP Based on 2009(bn)'})
    start_period = gdp.loc[gdp['Q'] == '2000q1', :].index[0] 
    #the index attribute gives an object, use index 0 to call the index number
    gdp_target = gdp.iloc[start_period:]
    return gdp_target

In [7]:
get_gdp()

,Q,Quarterly GDP(bn),GDP Based on 2009(bn)
212,2000q1,10031.0,12359.1
213,2000q2,10278.3,12592.5
214,2000q3,10357.4,12607.7
215,2000q4,10472.3,12679.3
216,2001q1,10508.1,12643.3
217,2001q2,10638.4,12710.3
218,2001q3,10639.5,12670.1
219,2001q4,10701.3,12705.3
220,2002q1,10834.4,12822.3
221,2002q2,10934.8,12893.0


In [16]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3
    For this assignment, only look at GDP data from the first quarter of 2000 onward.'''
    
    gdp = pd.read_excel("City data/gdplev.xls", skiprows = 7, usecols = [4,5,6]).rename(
        columns = {'Unnamed: 4':'Q', 
                   'Unnamed: 5':'Quarterly GDP(bn)', 
                   'Unnamed: 6':'GDP Based on 2009(bn)'})
    start_period = gdp.loc[gdp['Q'] == '2000q1', :].index[0] 
    #the index attribute gives an object, use index 0 to call the index number
    gdp_target = gdp.iloc[start_period:]
    for i in range(2, len(gdp_target)):
        if gdp_target.iloc[i, 2] < gdp_target.iloc[i-1, 2] and gdp_target.iloc[i-1, 2] < gdp_target.iloc[i-2, 2]:
            return gdp_target.iloc[i-1, 0], i
    return gdp_target

In [17]:
get_recession_start()

('2008q3', 35)

In [18]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel("City data/gdplev.xls", skiprows = 7, usecols = [4,5,6]).rename(
        columns = {'Unnamed: 4':'Q', 
                   'Unnamed: 5':'Quarterly GDP(bn)', 
                   'Unnamed: 6':'GDP Based on 2009(bn)'})
    start_period = gdp.loc[gdp['Q'] == '2000q1', :].index[0] 
    #the index attribute gives an object, use index 0 to call the index number
    gdp_target = gdp.iloc[start_period:]
    for i in range(35, len(gdp_target)):
        if gdp_target.iloc[i, 2] > gdp_target.iloc[i-1, 2] and gdp_target.iloc[i-1, 2] > gdp_target.iloc[i-2, 2]:
            return gdp_target.iloc[i-1, 0], i
       
    return 

In [19]:
get_recession_end()

('2009q3', 39)

In [20]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel("City data/gdplev.xls", skiprows = 7, usecols = [4,5,6]).rename(
        columns = {'Unnamed: 4':'Q', 
                   'Unnamed: 5':'Quarterly GDP(bn)', 
                   'Unnamed: 6':'GDP Based on 2009(bn)'})
    start_period = gdp.loc[gdp['Q'] == '2000q1', :].index[0] 
    #the index attribute gives an object, use index 0 to call the index number
    gdp_target = gdp.iloc[start_period:]
    recession_bot = float('inf')
    for i in range(35, 40):
        if gdp_target.iloc[i, 2] < recession_bot:
            recession_bot = gdp_target.iloc[i, 2]
    
    return gdp_target.loc[gdp['GDP Based on 2009(bn)'] == recession_bot, 'Q']

In [21]:
get_recession_bottom()

249    2009q2
Name: Q, dtype: object

In [39]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing = pd.read_csv("City data/City_Zhvi_AllHomes.csv", 
                          usecols = [1,2] + [i for i in range(51, 251)]).set_index(
                                                            ['State', 'RegionName'])
    quarters = {'01': 'q1', '04': 'q2', '07': 'q3', '10': 'q4'}
    for i in range(0, len(housing.columns), 3):
        year = housing.iloc[:, i:i+3].columns[0][:-3]
        month = housing.iloc[:, i:i+3].columns[0][-2:]
        housing['%s%s' % (year, quarters[month])] = np.mean(housing.iloc[:, i:i+3], axis = 1)
        
    housing_target = housing.iloc[:, 201:].reset_index().set_index(['State']).rename(index = states,
                            columns = {'RegionName': 'Region Name'}).set_index(['Region Name'], append = True)
    #return housing.head(15).filter(like = '2000')
    return housing_target

In [40]:
convert_housing_data_to_quarters()

,,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,2002q3,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,Region Name,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,5.872000e+05
California,Los Angeles,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,2.870000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,4.583889e+05
Illinois,Chicago,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,1.789667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,1.874667e+05
Pennsylvania,Philadelphia,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,6.216667e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,1.034667e+05
Arizona,Phoenix,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,1.305333e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,1.674111e+05
Nevada,Las Vegas,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,1.523333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,1.775000e+05
California,San Diego,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,3.311000e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,4.341333e+05
Texas,Dallas,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,9.540000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,1.276889e+05
California,San Jose,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,4.732333e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,6.728889e+05


In [61]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    house = convert_housing_data_to_quarters()
    gdp = get_gdp()
    university_towns = get_list_of_university_towns().set_index(['State', 'Region Name'])
    recession_periods = ['2008q3','2009q2'] #start at 2008q3, hit bottom at 2009q2
    #university_towns = pd.merge(left = university_towns, 
    #                            right = house, how = 'left',
    #                           on = ['State', 'Region Name']).set_index(
    #                                ['State', 'Region Name']).loc[:, 
    #                                    recession_periods].dropna(how = 'any')
    # Note: merge is for 2 dataframes to be combined together with column items
    # If combine based on index, apply join instead
    university_towns = university_towns.join(house[recession_periods]).dropna()
    non_university_towns = house.drop(university_towns.index).loc[:, recession_periods].dropna()
    university_towns['Diff'] = university_towns['2008q3'] - university_towns['2009q2']
    non_university_towns['Diff'] = non_university_towns['2008q3'] - non_university_towns['2009q2']
    #still have nan values in both dataframes
    #run t-test
    ttest = ttest_ind(university_towns['Diff'], non_university_towns['Diff'])
    if ttest[1] < 0.01:
        result = True
    else:
        result = False
    if university_towns['Diff'].mean() / university_towns['2008q3'].mean() < \
        non_university_towns['Diff'].mean()/non_university_towns['2008q3'].mean():
        lower = 'university towns'
    else:
        lower = 'non-university towns'
    
    #print(university_towns['Diff'].mean() / university_towns['2008q3'].mean(), non_university_towns['Diff'].mean()/non_university_towns['2008q3'].mean())
    return (result, ttest[1], lower)

In [62]:
run_ttest()

(True, 0.0043252148535112009, 'university towns')